In [1]:
import pandas as pd
import numpy as np
import os
from os import listdir
from os.path import isfile, join

In [8]:
datapath = "/Volumes/Dropbox (RajLab)/FateMap/FINAL_REVISION/analysis_Jonas/data/e_dist"
resultPath = "/Volumes/Dropbox (RajLab)/FateMap/FINAL_REVISION/analysis_Jonas/data/e_distResults"

In [9]:
folders = [x[0] for x in os.walk(path)]
folders = sorted(folders)

In [10]:
folders

['/Volumes/Aorta/analysis_Jonas/data/e_dist',
 '/Volumes/Aorta/analysis_Jonas/data/e_dist/fm01_1',
 '/Volumes/Aorta/analysis_Jonas/data/e_dist/fm01_2',
 '/Volumes/Aorta/analysis_Jonas/data/e_dist/fm06_1',
 '/Volumes/Aorta/analysis_Jonas/data/e_dist/fm06_2']

In [11]:
numberSimulFM01 = {}
numberSimulFM06 = {}
dfNumberClusters = pd.DataFrame()
dfDistances = pd.DataFrame()
for folder in folders:
    os.chdir(folder)
    files = [f for f in listdir(folder) if isfile(join(folder, f)) if ".DS_Store" not in f if ".csv" in f if "._" not in f]
    for file in files:
        dataset = folder.split("/")[-1].split("_")[0]
        sample = folder.split("/")[-1].split("_")[1]
        resolution = file.split("res")[1].split("_")[0]
        simul = file.split(".csv")[0].split("_")[-1]
        df = pd.read_csv(join(folder, file), index_col=0)
        arr = df.values
        matrix = np.matrix(arr)
        lower_diag = np.tril(matrix, k=-1)
        values = lower_diag[lower_diag > 0]
        dataset_list = [dataset] * len(values)
        sample_list = [sample] * len(values)
        numberCluster_list = [len(df)]* len(values)
        simul_list = [simul] * len(values)

        if dataset == "fm01":
            if len(df) in numberSimulFM01:
                numberSimulFM01[len(df)] = numberSimulFM01[len(df)] +1
            else:
                numberSimulFM01[len(df)] = 1
            numberSimul = numberSimulFM01
        else:
            if len(df) in numberSimulFM06:
                numberSimulFM06[len(df)] = numberSimulFM06[len(df)] +1
            else:
                numberSimulFM06[len(df)] = 1
            numberSimul = numberSimulFM06

        idList = [numberSimul[len(df)]] * len(values)
        dfDistances = pd.concat([dfDistances,pd.DataFrame({"Dataset": dataset_list, "sample": sample_list, "simulation": simul_list,"id": idList, "numberCluster": numberCluster_list, "distances": values})], ignore_index=True)
        dfNumberClusters = pd.concat([dfNumberClusters, pd.DataFrame({"Dataset": [dataset], "sample": [sample], "simulation":[simul], "resolution": [resolution], "numberCluster": [len(df)]})], ignore_index=True)

In [12]:
dfDistances.to_csv(join(resultPath, "Edistances_fm01VSfm06.csv"), index= False)

In [13]:
dfNumberClusters.to_csv(join(resultPath,"NumberClusters_fm01VSfm06.csv"), index=False)